In [1]:
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation, Dropout, Softmax
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pretty_midi

from data_preprocessing import get_train_data
from data_preprocessing import group_cqt_frames

from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import BinaryCrossentropy

In [2]:
# ACQUIRE DATA USING PICKLE
with open('../databases/pickled-preprocessed-databases/combined_database.pickle', 'rb') as f:
    cqt, labels = pickle.load(f)

print(np.shape(cqt))
print(np.shape(labels))

cqt_train, cqt_test, label_train, label_test = train_test_split(cqt, labels, test_size=0.2)
print(np.shape(cqt_train))
print(np.shape(cqt_test))
print(np.shape(label_train))
print(np.shape(label_test))

(867965, 88, 7)
(867965, 88)
(694372, 88, 7)
(173593, 88, 7)
(694372, 88)
(173593, 88)


In [3]:
model = models.Sequential()
# model.add(layers.Conv2D(50, (25, 5), activation='tanh', kernel_initializer='HeNormal', input_shape=(88, 7, 3)))
model.add(layers.Conv2D(50, (25, 5), activation='tanh', kernel_initializer='HeNormal', input_shape=(88, 7, 1)))
model.add(layers.MaxPooling2D((3, 1)))
model.add(Dropout(0.3))
model.add(layers.Conv2D(50, (5, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 1)))
model.add(Dropout(0.3))

model.add(layers.Flatten())
model.add(layers.Dense(1000, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(layers.Dense(200, activation="sigmoid"))
model.add(Dropout(0.3))
model.add(Dense(88))
model.add(Activation('sigmoid'))

model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9, learning_rate=0.01),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.F1Score()])
# model.summary()

/opt/homebrew/anaconda3/envs/tensorflow_env/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [4]:
# checkpoint_filepath = '/kaggle/working/full_model_checkpoint.weights.h5'
# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_weights_only=True,
#     monitor='val_accuracy',
#     mode='max',
#     save_best_only=False,
#     save_freq="epoch")

In [5]:
model.load_weights("../saved-models/full-models-from-zero/full_model_from_zero_6.weights.h5")

/opt/homebrew/anaconda3/envs/tensorflow_env/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 12 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [ ]:
history = model.fit(
    x=cqt_train,
    y=label_train,
    batch_size=32,
    epochs=100,
    validation_data=(cqt_test, label_test),
    shuffle=True)
#     callbacks=[model_checkpoint_callback])